In [3]:
import pandas as pd
import numpy as nm

In [32]:
# read the data
d1=pd.read_excel('D:\GoogleDrive\custom Codes\Mar18\Collateral\Loan_Data2.xlsx')
# see the dimension
d1.shape

(98179, 40)

In [33]:
# bring customer group id where customer id is not present
d1['CUSTOMER_ID']=d1['CUSTOMER_ID'].fillna(d1['CUSTOMER_GROUP_ID'])

# turn the customer id into string
d1['CUSTOMER_ID']=d1['CUSTOMER_ID'].astype(str)

# make the leading zero and turn customer id into 10 digits
d1['CUSTOMER_ID']=d1['CUSTOMER_ID'].apply(lambda x: x.zfill(12)).str[:10]
#d1.head()

In [34]:
# make unique id by concating branch id and collateral id
d1['UI']=d1['BRANCH_ID'].astype(str)+d1['COLLATERAL_ID'].astype(str)

In [35]:
# drop duplicate rows by the new column UI
d1=d1.drop_duplicates(subset='UI', keep="first")
d1.shape

(55745, 41)

In [36]:
# fill the na is eligible collateral value with 0
d1['ELIGIBLE_SECURITY_VALUE']=d1['ELIGIBLE_SECURITY_VALUE'].fillna(0)

In [37]:
# now group by sum for each customer id
do=d1.groupby(['CUSTOMER_ID'])['ELIGIBLE_SECURITY_VALUE'].agg(sum)
do=do.reset_index()
do.head()
do['ELIGIBLE_SECURITY_VALUE'].sum()

183500370904.08997

In [38]:
# now the data with actual provision
d2=pd.read_excel('D:\GoogleDrive\custom Codes\Mar18\Collateral\Loan_D1.xlsx')
#d2.columns

In [39]:
d2['CUSTOMER_ID']=d2['CUSTOMER_ID'].fillna(d2['CUSTOMER_GROUP_ID'])
d2['CUSTOMER_ID']=d2['CUSTOMER_ID'].astype(str).apply(lambda x: x.zfill(12))
d2['CUSTOMER_ID']=d2['CUSTOMER_ID'].str[:10]

In [40]:
#do.head()

In [41]:
# customer id wise total outstanding
du=d2.groupby(['CUSTOMER_ID'])['CLOSE_BALA_LCY'].agg(sum).reset_index()

# rename columns
du=du.rename(columns={'CLOSE_BALA_LCY':'TOTAL_OUTSTANDING'})
#du.head()

In [43]:
print(du['TOTAL_OUTSTANDING'].sum())
print(du.shape)

102567357006.81999
(68085, 2)


In [44]:
# select the two columns
dn=d2[['CUSTOMER_ID', 'CLOSE_BALA_LCY']]

#mege the actual outstanding with the total outstanding per customer id
dt=pd.merge(dn, du,  on='CUSTOMER_ID', how='left')
dt.shape
dt['TOTAL_OUTSTANDING'].sum()
dt.head()
print(dt['CLOSE_BALA_LCY'].sum())

102567357006.82


In [45]:
# merge total collateral data with total outstanding and actual outstanding
dm=pd.merge(dt, do, on='CUSTOMER_ID', how='left')
dm.shape

(76098, 4)

In [46]:
# now calculate collateral value for each account (based on its outstanding)
dm['COL_PER_AC']=dm['ELIGIBLE_SECURITY_VALUE']*dm['CLOSE_BALA_LCY']/dm['TOTAL_OUTSTANDING']
dm.head()
dm['COL_PER_AC'].sum()

175834104542.96997